# Computational Theory Tasks

This notebook contains the solutions to the tasks for the Computational Theory module.

## Task 1: Binary Representations

This task implements various binary manipulation functions as specified in the requirements.

In [3]:
import unittest

def rotl(x, n=1):
    """Rotate bits in a 32-bit unsigned integer to the left by n places."""
    n %= 32  # Ensure n is within valid range
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

def rotr(x, n=1):
    """Rotate bits in a 32-bit unsigned integer to the right by n places."""
    n %= 32  # Ensure n is within valid range
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

def ch(x, y, z):
    """Choose bits from y where x has bits set to 1, otherwise take bits from z."""
    return (x & y) | ((~x & 0xFFFFFFFF) & z)

def maj(x, y, z):
    """Majority function: output has a 1 where at least two of x, y, and z have 1's in that position."""
    return (x & y) | (x & z) | (y & z)

### Testing the binary functions

In [ ]:
# Test cases for rotl function
print("Testing rotl function:")
test_value = 0b10110011
result = rotl(test_value, 2)
print(f"rotl(0b{test_value:08b}, 2) = 0b{result:08b}")
print(f"Expected: 0b11001101")
print(f"Correct: {result == 0b11001101}")
print()

# Test cases for rotr function
print("Testing rotr function:")
test_value = 0b10110011
result = rotr(test_value, 2)
print(f"rotr(0b{test_value:08b}, 2) = 0b{result:08b}")
print(f"Expected: 0b11101100")
print(f"Correct: {result == 0b11101100}")
print()

# Test cases for ch function
print("Testing ch function:")
x, y, z = 0b10110011, 0b11001100, 0b11110000
result = ch(x, y, z)
print(f"ch(0b{x:08b}, 0b{y:08b}, 0b{z:08b}) = 0b{result:08b}")
print(f"Expected: 0b11001100")
print(f"Correct: {result == 0b11001100}")
print()

# Test cases for maj function
print("Testing maj function:")
x, y, z = 0b10110011, 0b11001100, 0b11110000
result = maj(x, y, z)
print(f"maj(0b{x:08b}, 0b{y:08b}, 0b{z:08b}) = 0b{result:08b}")
print(f"Expected: 0b11110000")
print(f"Correct: {result == 0b11110000}")